In [ ]:
'''
This notebook evaluates prediction results made with the GRETEL model.
- NOTE: Prediction and evaluation is performed in two separate notebooks. This notebook is for evaluation and needs to run in a geo environment (env_geo)
- Specify file containing prediction
The notebook will load the prediction results, evaluate and plot evaluation metrics
Optionally, a map containing prediction and ground truth is plotted
'''

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import warnings
import pickle
import sys
import json
import neptune
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
sys.path.append('../datawrangling')
# import modules
import visualize
import geometry_utils
import dataloader_geo
from maritime_traffic_network import MaritimeTrafficNetwork
import prediction_model_evaluation_metrics as metrics
from make_trajectories_from_AIS import add_ship_metadata

In [ ]:
# Specify path to directory of model
location = 'oslo'
path_format = 'node2node'
directory = '../../models/gretel_prediction_models/'+location+'_passenger/'+path_format+'/'

# Specify file that contains prediction results
prediction_task = 'next_nodes'
n_steps = 10

# Specify evaluation mode
eval_mode = 'path'

In [ ]:
# load metadata from file
with open(directory+'/metadata_'+prediction_task+str(n_steps)+'.json', 'r') as json_file:
    meta_dict = json.load(json_file)
meta_dict

In [ ]:
# retrieve parameters for prediction
prediction_task = meta_dict['prediction_task']
n_start_nodes= meta_dict['n_start_nodes']
n_steps = meta_dict['n_steps']
network_name = meta_dict['network_name']
test_dates = literal_eval(meta_dict['test_dates'])
filter = meta_dict['filter']

In [ ]:
# load maritime traffic network
network_path = '../../models/networks/best_networks/' + network_name + '.obj'
fileObj = open(network_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

In [ ]:
# Load test trajectories from file
traj_path_prefix = '../../data/processed/'
test_trajectories = dataloader_geo.load_trajectories(traj_path_prefix, location, network.crs, test_dates)

In [ ]:
# load predictions from file
predictions = pd.read_csv(directory+'/predictions_'+prediction_task+str(n_steps)+'.csv')
predictions['ground_truth'] = predictions['ground_truth'].apply(literal_eval)
predictions['prediction'] = predictions['prediction'].apply(literal_eval)

In [ ]:
# check if all predictions are valid
for index, row in predictions.iterrows():
    if geometry_utils.is_valid_path(network.G, row['ground_truth']) == False:
        print(row['mmsi'].tolist())

In [ ]:
# Evaluate
evaluation_results, fig = metrics.evaluate_given_predictions(prediction_task, predictions, test_trajectories, 
                                                             network, n_start_nodes=n_start_nodes, n_steps=n_steps, 
                                                             eval_mode=eval_mode)
nan_mask = evaluation_results.isna().any(axis=1)
failure_rate = nan_mask.sum() / len(evaluation_results)
mean_abs_err = np.mean(evaluation_results[~nan_mask]["SSPD"])
median_abs_err = np.median(evaluation_results[~nan_mask]["SSPD"])
choice_accuracy = np.mean(evaluation_results[~nan_mask]["choice_accuracy"])

In [ ]:
'''
# Log experiment with neptune
run = neptune.init_run(
    project="jandrik91/RoutePredictions",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxYmQzMjgwZS1jZGYwLTQ2YjktYWNjOS02MjBlZWEzNzUzNDcifQ==",
)  # your credentials

# evaluate
evaluation_results, fig = metrics.evaluate_given_predictions(prediction_task, predictions, test_trajectories, 
                                                             network, n_start_nodes=n_start_nodes, n_steps=n_steps, 
                                                             eval_mode=eval_mode)

nan_mask = evaluation_results.isna().any(axis=1)
failure_rate = nan_mask.sum() / len(evaluation_results)
mean_abs_err = np.mean(evaluation_results[~nan_mask]["SSPD"])
median_abs_err = np.median(evaluation_results[~nan_mask]["SSPD"])
choice_accuracy = np.mean(evaluation_results[~nan_mask]["choice_accuracy"])

# save experiment
run["network_name"] = network_name
run["n_points"]=len(network.gdf)
run["n_nodes"]=network.G.number_of_nodes()
run["n_edges"]=network.G.number_of_edges()

params = network.hyperparameters
params['clustering_metric_V_coord'] = params['clustering_metric_V'][0][0]
params['clustering_metric_V_cog'] = params['clustering_metric_V'][2][2]
params['clustering_metric_V_speed'] = params['clustering_metric_V'][4][4]
run["network_parameters"] = params

run["training_data"] = {'training_dates':meta_dict['training_dates'],
                        'n_training_paths':meta_dict['n_training_paths']}

run["test_data"] = {'test_dates':str(test_dates),
                    'selection_start':meta_dict['selection_start'],
                    'selection_end':meta_dict['selection_end'],
                    'selection_step':meta_dict['selection_step'],
                    'n_test_paths':meta_dict['n_test_paths']}

run["prediction_task"] = prediction_task
run["eval_mode"] = eval_mode
run["model_type"] = meta_dict['model_type']
run["n_start_nodes"] = n_start_nodes
run["n_steps"] = n_steps

run['node_features'] = meta_dict['node_features']
run['egde_features'] = meta_dict['egde_features']
run['path_format'] = meta_dict['path_format']
run['lr'] = meta_dict['lr']
run['loss'] = meta_dict['loss']
run['n_epochs'] = meta_dict['n_epochs']
run['MOGen_n_walks'] = meta_dict['n_walks']
run['target_prediction'] = meta_dict['target_prediction']

run["plot"].upload(fig)
run["failure_rate"] = failure_rate
run["mean_abs_err"] = mean_abs_err
run["median_abs_err"] = median_abs_err
run["choice_accuracy"] = choice_accuracy
run["filter"] = filter

run.stop()
'''

In [ ]:
# Plot a prediction against the ground truth
i = 10   # choose example from test data
mmsi = evaluation_results['mmsi'].iloc[i]
predictions = {evaluation_results['predicted_path'].iloc[i] : 1}
start_node = [evaluation_results['predicted_path'].iloc[i][0]]
trajectory = test_trajectories.get_trajectory(mmsi)
true_path = evaluation_results['true_path'].iloc[i]
map = visualize.map_prediction_and_ground_truth(predictions, start_node, trajectory, true_path, network, 
                                                min_passages=5, opacity=0.2, location=location)
map

In [ ]:
# Plot performance by ship category
# add original mmsi column back to dataframe
evaluation_results.rename(columns={'mmsi':'id'}, inplace=True)
evaluation_results['mmsi'] = evaluation_results['id'].str[:9].astype(int)

# add metadata to each mmsi
meta_file = '../../data/external/seilas-2022.csv'
evaluation_results_meta = add_ship_metadata(meta_file, evaluation_results)

# get choice accuracy and SSPD by shipgroup
sspd_by_group = evaluation_results_meta.groupby(['skipsgruppe'])['SSPD'].mean()
cacc_by_group = evaluation_results_meta.groupby(['skipsgruppe'])['choice_accuracy'].mean()

x = cacc_by_group.values
y = sspd_by_group.values
categories = sspd_by_group.index

fig, ax = plt.subplots(figsize=[6, 4])

# Scatter plot with different colors and markers for each category
scatter = ax.scatter(x, y, c=range(len(categories)), cmap='viridis', marker='o', s=100)

# Annotate each point with the ship category name
for i, category in enumerate(categories):
    ax.annotate(category, (x[i], y[i]), textcoords="offset points", xytext=(5,5), ha='left')

max_x_value = max(x)
max_y_value = max(y)
ax.set_xlim(0.4, max_x_value+0.05)
ax.set_ylim(0, max_y_value+100)

ax.set_xlabel('$CACC$')
ax.set_ylabel('$MD_{SSPD}$ (m)')
plt.title('Prediction performance by Shipgroup')
plt.savefig('prediction_performance_by_ship_category_tromso_Gretel_3nodes.pdf')

plt.show()